In [1]:
import sys
sys.path.append("../../src/earthtext")

from osm import osm
import os
from progressbar import progressbar as pbar
import shapely as sh 
from pyproj import CRS
import numpy as np
epsg4326 = CRS.from_epsg(4326)
import geopandas as gpd
import pandas as pd
from importlib import reload
from rlxutils import Command
import rlxutils
reload(osm)

<module 'osm.osm' from '/home/ubuntu/earth-text/notebooks/naip/../../src/earthtext/osm/osm.py'>

# partition osm objects from origin grid to destination grid

osm objects have already been partition into `origin_grid`

origin grid is much coarser than destination grid to make more efficient the intersection of osm geometries for each chip

## setup origin grid



In [2]:
orig_partsdir = "/opt/data/osm/california-parts/"
dest_partsdir = "/opt/data/california-naip-chips/osm"
os.makedirs(dest_partsdir, exist_ok=True)
chip_ids_pbf = [i.split(".")[0] for i in os.listdir(orig_partsdir) if i.endswith(".pbf")]
chip_ids_geojson = [i.split(".")[0] for i in os.listdir(orig_partsdir) if i.endswith(".geojson")]

if len(set(chip_ids_pbf).intersection(set(chip_ids_geojson)))!=len(chip_ids_pbf)!=len(chip_ids_geojson):
    raise ValueError("missing chips in geojson o pbf")

In [3]:
orig_grid = []
for chip_id in chip_ids_geojson:
    with open(f"{orig_partsdir}/{chip_id}.geojson") as f:
        orig_grid.append([chip_id, sh.from_geojson(f.read())])

orig_grid = gpd.GeoDataFrame(pd.DataFrame(orig_grid, columns=['chip_id', 'geometry']), crs=epsg4326)
orig_grid.shape

(984, 2)

In [4]:
orig_grid.head()

,chip_id,geometry
0,000235e893b24,"POLYGON ((-121.87715 39.14450, -121.65850 39.1..."
1,0002c308d55fc,"POLYGON ((-114.92112 35.05498, -114.92112 35.2..."
2,0018e8bbce095,"POLYGON ((-120.25595 36.95832, -120.55074 36.9..."
3,003fa84955512,"POLYGON ((-119.13010 35.78528, -119.40375 35.7..."
4,004aaa9df44f2,"POLYGON ((-124.14191 40.08352, -124.24849 40.0..."


## setup destination grid

In [5]:
dest_grid = gpd.read_parquet("/opt/data/california-naip-chips/california-naip-chips-200k-raw.parquet")
dest_grid.head()

,date,chip_index_x,chip_index_y,cloud_cover_percentage,nodata_percentage,geometry,embeddings,original_chip_id
chip_id,,,,,,,,
3416f3c464df4,2022-05-06,27,5,0.0,0.0,"POLYGON ((-116.89310 33.99492, -116.89310 33.9...","[-0.13333216, 0.0399988, 0.14124717, 0.0125191...",ca_m_3311601_ne_11_060_20220506-27-5
22fdfb499b95a,2022-05-06,24,24,0.0,0.0,"POLYGON ((-116.89812 33.96860, -116.89812 33.9...","[-0.1470401, 0.026783561, 0.12910064, 0.014626...",ca_m_3311601_ne_11_060_20220506-24-24
06e39238c16ee,2022-05-06,19,6,0.0,0.0,"POLYGON ((-116.90640 33.99355, -116.90641 33.9...","[-0.14139079, 0.021682426, 0.14206946, 0.00693...",ca_m_3311601_ne_11_060_20220506-19-6
108471e507d4c,2022-05-06,30,0,0.0,0.0,"POLYGON ((-116.88810 34.00184, -116.88810 34.0...","[-0.10461623, 0.033646498, 0.115653, 0.0179855...",ca_m_3311601_ne_11_060_20220506-30-0
16dd360b58af6,2022-05-06,29,24,0.0,0.0,"POLYGON ((-116.88981 33.96860, -116.88981 33.9...","[-0.13561213, 0.03594061, 0.12552005, 0.009031...",ca_m_3311601_ne_11_060_20220506-29-24


In [6]:
orig_grid.shape, dest_grid.shape

((984, 2), (198919, 8))

## get coverage of destination grid

In [7]:
gcov = dest_grid.geometry.values[0]
for gi in pbar(dest_grid.geometry.values):
    gcov = gcov.union(gi).convex_hull

100% (198919 of 198919) |################| Elapsed Time: 0:00:08 Time:  0:00:080000


## subset origin grid only to destination grid coverage

In [8]:
orig_grid = orig_grid[[gi.intersects(gcov) for gi in orig_grid.geometry.values]]

In [ ]:
orig_grid.explore()

## check distribution of destination grid chips in a couple of sample origin chips

In [ ]:
_orig_grid = orig_grid.sample(2)
c = sh.geometry.GeometryCollection(_orig_grid.geometry.values[1:])
dg = dest_grid[[i.intersects(c) for i in dest_grid.geometry.values]]
#pd.concat([dg, _orig_grid])[['geometry']].explore()
dg[['geometry']].explore()


## partition osm objects from origin grid to destination grid

In [14]:
notfound = []
dest_chips_withnodata = []
dest_chips_withdata = []
# loop over coarse grained geometries (origin)
for count, (_, orig_row) in enumerate(orig_grid.iloc[46+27:].iterrows()):
    et = rlxutils.ElapsedTimes()
    orig_row_geom = orig_row.geometry
    dg = dest_grid[[i.intersects(orig_row_geom) for i in dest_grid.geometry.values]]
    print (f"{count}/{len(orig_grid)}", orig_row.chip_id, 'dest_chips', len(dg), flush=True)

    # loop over fine grained geometries (dest). 
    parquets_cache = {}
    for chip_id, dest_row in pbar(dg.iterrows(), max_value=len(dg)):
        
        dest_row_geom = dest_row.geometry
        
        dest_parquet = f"{dest_partsdir}/{chip_id}.parquet"
        if os.path.isfile(dest_parquet):
            continue

        # dest geometry might intersect several origin geometry, 
        # not only the one in the outer loop
        with et("ogintersect"):
            og = orig_grid[[i.intersects(dest_row_geom) for i in orig_grid.geometry.values]]
        
        if len(og) == 0:
            continue

        # loop over all intersecting coarse geometries and
        # get all osm objects
        dgdata = []
        for chip_id in og.chip_id.values:
            if chip_id in parquets_cache.keys():
                p = parquets_cache[chip_id]
            else:
                with et("read_parquet"):
                    fname = f"{orig_partsdir}/{chip_id}.parquet"
                    if not os.path.isfile(fname):
                        notfound.append(fname)
                        continue
                    p = gpd.read_parquet(fname)
                    parquets_cache[chip_id]=p

            # only get the parts of osm geometries within this finer grain geometry
            with et("dgintersect"):
                pi = p[p.intersects(dest_row_geom)].copy()
                pi['geometry'] = [gi.intersection(dest_row_geom) for gi in pi.geometry.values]
            dgdata.append(pi)
        dgdata = pd.concat(dgdata)
        if len(dgdata)==0:
            dest_chips_withnodata.append(chip_id)
            continue
        dest_chips_withdata.append(chip_id)
        with et("save"):
            dgdata.to_parquet(dest_parquet)


0/132 21706a5a16e06 dest_chips 158


100% (158 of 158) |######################| Elapsed Time: 0:00:04 Time:  0:00:040:00


1/132 2201d9aa51bef dest_chips 1593


100% (1593 of 1593) |####################| Elapsed Time: 0:08:49 Time:  0:08:490547


2/132 222b87d3538c6 dest_chips 2256


...

100% (2882 of 2882) |####################| Elapsed Time: 0:00:25 Time:  0:00:250001


57/132 373b0b28ea85a dest_chips 2562


100% (2562 of 2562) |####################| Elapsed Time: 0:00:47 Time:  0:00:470004


58/132 3811c01719920 dest_chips 1937


100% (1937 of 1937) |####################| Elapsed Time: 0:00:11 Time:  0:00:110001


In [ ]:
len(dgdata)